# WEkEO and xCube Viewer

This Notebook shows a workflow of using the (xCube)[https://xcube.readthedocs.io/en/latest/index.html] python package to interact, visualize and share datacubes which are created using Copernicus data. It should display the potential of the concept of datacubes for Copernicus data for an improved user experience and future implementations of Serverless functions. 

There are several wayt to create datacubes. Obe way for a wide range of datasets from the Copernicus services is the usage of CliMetLab and xArray. Other, non-gridded data, such as satellite data coming from the Sentinel satellites can be preprocesses using SNAP or the data-tailor to generate datacubes. Other types of datacubes are already preprocessed by third parties and available to the public for use. 

xCube and especially the xCube Viewer is a powerful tool to interact with datacubes from different sources at the same time and share data and insights with others. 

# Contents

The notebook is structured as follows: 


- [Installation of packages](#Installation-of-packages)
- [Importing Data Cubes](#Importing-Data-Cubes)
- [xCube Viewer inside a local Jupyter Notebook](#xCube-Viewer-inside-a-local-Jupyter-Notebook)
- [xCube Viewer on the VM](#xCube-Viewer-on-the-VM)


# Installation of packages


In [ ]:
#!git clone https://github.com/dcs4cop/xcube.git
#!cd xcube
#!pip install --no-deps --editable .

In [ ]:
#!pip install xcube_jl_ext

# Importing Data Cubes

In this example only datacubes from the local file system are accessed. There is also the possibility to import datacubes which are located on a s3 bucket or are located in preconfigures data stores (e.g. CCI or SentinelHub).  

In [1]:
# mandatory xcube store imports
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from xcube.core.select import select_subset

# Utilities for notebook visualization. 
# JSON provides pretty-printing of Python lists and dictionaries.
from IPython.display import JSON

import xarray as xr

## Import the LST Datacube created from CliMetLab

In [2]:
lst_germany = xr.open_zarr("/home/annalenae/Demo_CML_xCube/lst_germany.zarr")
lst_germany

<xarray.Dataset>
Dimensions:  (time: 47, lat: 202, lon: 247)
Coordinates:
  * lat      (lat) float64 55.98 55.94 55.89 55.85 ... 47.14 47.1 47.05 47.01
  * lon      (lon) float64 5.0 5.045 5.089 5.134 ... 15.85 15.89 15.94 15.98
  * time     (time) datetime64[ns] 2021-07-01 ... 2021-07-02T23:00:00
Data variables:
    LST      (time, lat, lon) float32 dask.array<chunksize=(1, 202, 247), meta=np.ndarray>
Attributes: (12/31)
    Conventions:                CF-1.6
    algorithm_version:          HIMAWARI8-LST_v4.1.0; GOES16-LST_v4.1.0; MSG4...
    archive_facility:           VITO
    comment:                    Land Surface Temperature (LST) is the radiati...
    contacts:                   https://land.copernicus.eu/global/contact
    copyright:                  Copernicus Service Information 2021
    ...                         ...
    references:                 https://land.copernicus.eu/global/products/lst
    sensor:                     AHI, ABI, SEVIRI, SEVIRI
    source:                     Data was derived from satellite imagery.
    time_coverage_end:          2021-07-01T00:30:00Z
    time_coverage_start:        2021-06-30T23:30:00Z
    title:                      LST Germany 2021-07

In [3]:
t2m_germany = xr.open_zarr("/home/annalenae/Demo_CML_xCube/t2m_germany.zarr")

## Import Sentinel 3 OLCI Data Cube  

In [4]:
cube_s3_baltic = xr.open_zarr('/home/annalenae/Demo_CML_xCube/s3chl_baltic.zarr')

In [5]:
cube_s3_baltic

<xarray.Dataset>
Dimensions:  (time: 20, lat: 281, lon: 491)
Coordinates:
  * lat      (lat) float64 54.01 54.02 54.04 54.05 ... 57.95 57.96 57.98 57.99
  * lon      (lon) float64 15.01 15.02 15.04 15.05 ... 21.95 21.96 21.98 21.99
  * time     (time) datetime64[ns] 2021-07-01T08:25:12.862935040 ... 2021-07-...
Data variables:
    chl_nn   (time, lat, lon) float64 dask.array<chunksize=(1, 281, 491), meta=np.ndarray>
Attributes: (12/48)
    Conventions:               CF-1.5
    EPCT_aggregation:          null
    EPCT_chain name:           baltic_projected
    EPCT_creation_time:        2023-08-31T10:01:42.306718Z
    EPCT_filter:               null
    EPCT_format:               netcdf4
    ...                        ...
    references:                S3IPF PDS 004.3 - i2r3 - Product Data Format S...
    resolution:                [ 1080 1176 ]
    source:                    IPF-OL-2 07.01
    start_time:                2020-07-06T08:25:12.862935Z
    stop_time:                 2020-07-06T09:09:35.329476Z
    title:                     CHL Baltic 2020-07

# xCube Viewer inside a local Jupyter Notebook

In [6]:
from xcube.webapi.viewer import Viewer

In [7]:
#setting the style variables

In [8]:
viewer = Viewer(server_config={
    "Styles": [
        {
            "Identifier": "LST",
            "ColorMappings": {
                "LST": {
                    "ValueRange": [250, 300],
                    "ColorBar": "inferno"
                }
            }
        }, 
        {
            "Identifier": "t2m",
            "ColorMappings": {
                "t2m": {
                    "ValueRange": [250, 300],
                    "ColorBar": "inferno"
                },
                "t2m_monthly": {
                    "ValueRange": [250, 300],
                    "ColorBar": "inferno"
                },
                "diff": {
                    "ValueRange": [-10, 10],
                    "ColorBar": "cividis"
                }
            }
        }, 
        {
            "Identifier": "CHL",
            "ColorMappings": {
                "chl_nn": {
                    "ValueRange": [0, 5],
                    "ColorBar": "viridis"
                }
            }
        },
    ]
})

In [9]:
viewer.add_dataset(t2m_germany, style="t2m")
viewer.add_dataset(lst_germany, style="LST")
viewer.add_dataset(cube_s3_baltic, style="CHL")

'd922ff23-cd80-43a2-8ed7-590d88e69e28'

In [10]:
viewer.info()

Server: http://localhost:8889/proxy/8000
Viewer: http://localhost:8889/proxy/8000/viewer/?serverUrl=http://localhost:8889/proxy/8000


In [11]:
viewer.show()

404 GET /viewer/config/config.json (127.0.0.1): xcube viewer has not been been configured
404 GET /viewer/config/config.json (127.0.0.1) 7.44ms


# xCube Viewer on the VM